In [3]:
from common import *
from matrix import *
from helper.weighted import *
%matplotlib inline
canvas(16, 12)

In [4]:
with open(top_dir + 'data/chance/days_order.txt', 'r') as file:
    days = pd.Series([int(x) for x in file.read().splitlines()], name='Day')
n_days = len(days)
days_a, days_b, days_c = days[:n_days // 3], days[n_days // 3:(2 * n_days) // 3], days[(2 * n_days) // 3:]
#days_a, days_b = set(days[:n_days // 2]), set(days[n_days // 2:])
tr, cv, ep = df.Day.isin(days_a), df.Day.isin(days_b), df.Day.isin(days_c)
#tr, cv = df.Day.isin(days_a), df.Day.isin(days_b)

In [ ]:
raise Stop

In [7]:
duse = dfh # <=============
rawfeats = x_feats # <============
def_matrices(globals(), duse, 10000)
for xname in rawfeats:
    X = duse.pivot('Day', 'Stock', xname)
    Xa = X.fillna(0)
    Xm = X.fillna(X.mean())
    Xpct = X.pct()
    Xpcta = Xpct.fillna(0)
    Xpctm = Xpct.fillna(0.5)
    for ext in ['', 'a', 'm', 'pct', 'pcta', 'pctm']:
        globals()[xname.upper() + ext] = locals()['X' + ext]

In [8]:
def mse(Y_, st):
    return (W[st] * (Y_ - Y[st]) ** 2).sum().sum() / W[st].sum().sum()
def rmse(Y_, st):
    return np.sqrt(mse(Y_, st))
def prss(Y_, st):
    return mse(Y_, st) / mse(0, st)
def lbss(Y_, st):
    return prss(Y_, st) * 0.500944
def metric(Y_, st):
    print('(' + ', '.join('{:.6f}'.format(round(func(Y_, st), 6)) for func in [rmse, prss, lbss]) + ')')

In [9]:
Xa = pd.concat({x: globals()[x.upper() + 'a'] for x in rawfeats}, axis=1)#.swaplevel(axis=1).sort_index(axis=1)
Xpcta = pd.concat({x: globals()[x.upper() + 'pcta'] for x in rawfeats}, axis=1)#.swaplevel(axis=1).sort_index(axis=1)

In [10]:
Xuse = Xpcta #.loc[:, (ss, 'x4')]

#### my alg

In [4]:
fname = 'x_.sgnint' # <==============
Xfeat = pd.read_hdf(top_dir + '/data/features/' + fname + '.hdf5')

In [5]:
keep = [i for i, s in enumerate(Xfeat.columns) if len(s) <= len('x_.sgnint(0000/)')] # <================
filt = index_nvdf(dh, 246) # <==============
Xfeat = Xfeat[~filt]
Xfeat = Xfeat.iloc[:, keep]

In [6]:
Xfeat = make_stock_wide(Xfeat)

In [7]:
#Xfeat.fillna(0, inplace=True)
Xfeat = Xfeat.pct()
Xfeat.fillna(0.5, inplace=True)
Xfeat *= 2
Xfeat -= 1

In [8]:
Ya = make_stock_wide(dh.y).fillna(0)
Wa = make_stock_wide(dh.Weight).fillna(0)

In [12]:
# ~246 filtered stocks
Xtest = Xfeat
Ytest = Ya
Wtest = Wa
tr, cv, ep = Ytest.index.isin(days_a), Ytest.index.isin(days_b), Ytest.index.isin(days_c)
alg(Xtest[tr], Ytest[tr], Wtest[tr], Xtest[cv], Ytest[cv], Wtest[cv], Xtest[ep], Ytest[ep], Wtest[ep], intercept=True)

0.785056613367 0.810735674348 0
()

[469]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........400.........410.........420.........430.........440.........450.........460........
0.283098951721 0.303078131181 1
(x_.sgnint(5/))

[468]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300

In [11]:
# ~246 filtered stocks
Xtest = Xfeat
Ytest = Ya
Wtest = Wa
tr, cv, ep = Ytest.index.isin(days_a), Ytest.index.isin(days_b), Ytest.index.isin(days_c)
alg(Xtest[tr], Ytest[tr], Wtest[tr], Xtest[cv], Ytest[cv], Wtest[cv], Xtest[ep], Ytest[ep], Wtest[ep], intercept=False)

0.785056613367 0.810735674348 0
()

[469]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........400.........410.........420.........430.........440.........450.........460........
0.270127143637 0.293438942211 1
(x_.sgnint(345/))

[468]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........3

In [14]:
# 246 filtered stocks
Xtest = Xfeat
Ytest = Ya
Wtest = Wa
tr, cv, ep = Ytest.index.isin(days_a), Ytest.index.isin(days_b), Ytest.index.isin(days_c)
alg(Xtest[tr], Ytest[tr], Wtest[tr], Xtest[cv], Ytest[cv], Wtest[cv], Xtest[ep], Ytest[ep], Wtest[ep], intercept=True)

0.785056613367 0.810735674348 0
()

[469]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........400.........410.........420.........430.........440.........450.........460........
0.538600058198 0.564115849009 1
(x_.sgnint(3/))

[468]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300

In [10]:
# 246 filtered stocks
Xtest = Xfeat
Ytest = Ya
Wtest = Wa
tr, cv, ep = Ytest.index.isin(days_a), Ytest.index.isin(days_b), Ytest.index.isin(days_c)
alg(Xtest[tr], Ytest[tr], Wtest[tr], Xtest[cv], Ytest[cv], Wtest[cv], Xtest[ep], Ytest[ep], Wtest[ep], intercept=False)

0.785056613367 0.810735674348 0
()

[469]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........400.........410.........420.........430.........440.........450.........460........
0.513674223186 0.54465370319 1
(x_.sgnint(34/5))

[468]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........30

In [18]:
# all stocks
Xtest = Xfeat
Ytest = Ya
Wtest = Wa
tr, cv, ep = Ytest.index.isin(days_a), Ytest.index.isin(days_b), Ytest.index.isin(days_c)
alg(Xtest[tr], Ytest[tr], Wtest[tr], Xtest[cv], Ytest[cv], Wtest[cv], Xtest[ep], Ytest[ep], Wtest[ep], intercept=True)

0.785056613367 0.810735674348 0
()

[469]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........400.........410.........420.........430.........440.........450.........460........
0.750460299529 0.752426836957 1
(x_.sgnint(03/5))

[468]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........3

In [17]:
# all stocks
Xtest = Xfeat
Ytest = Ya
Wtest = Wa
tr, cv, ep = Ytest.index.isin(days_a), Ytest.index.isin(days_b), Ytest.index.isin(days_c)
alg(Xtest[tr], Ytest[tr], Wtest[tr], Xtest[cv], Ytest[cv], Wtest[cv], Xtest[ep], Ytest[ep], Wtest[ep], intercept=False)

0.785056613367 0.810735674348 0
()

[469]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........400.........410.........420.........430.........440.........450.........460........
0.731774081146 0.751495624191 1
(x_.sgnint(0/5))

[468]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........30

#### small

In [ ]:
from sklearn.linear_model import LinearRegression as LR

def do_lr(X, Y, W):
    lr_list = []
    # assert the columns of the input can be indexed by range(0, 3023)
    for k in range(N_STOCK):
        x, y, w = X[k], Y[k], W[k]
        if w.sum() == 0:
            lr_list += [None]
            continue
        lr = LR(fit_intercept=True)
        # don't forget x1 and x2 have NaNs for valid y. try setting W=0 for when y and W does not exist and then filling in NaNs
        lr = lr.fit(x, y, w)
        lr_list += [lr]
    return lr_list

In [66]:
lr_list = do_lr(Xuse[tr], Ya[tr], Wa[tr])

In [51]:
Yx = np.stack([(lr.predict(Xuse[k]) if lr is not None else np.zeros(Y.shape[0]))
               for lr, k in zip(lr_list, range(N_STOCK))], axis=1)
Yx = Yx + Y * 0

In [52]:
st = cv
Yxm = Yx.where(Y[tr].count() > 50, Y[tr].mean().fillna(0), axis=1)

#### big

In [20]:
bigX = Xpcta.stack(level=0)
bigY = Ya.stack(level=0)
bigW = Wa.stack(level=0)

In [24]:
lr = LR()
lr = lr.fit(bigX, bigY, bigW)

In [28]:
Yx = np.stack([(lr.predict(Xpcta[k]) if lr is not None else np.zeros(Y.shape[0]))
               for k in range(N_STOCK)], axis=1)
Yxm = Yx + Y * 0

### metric test sees

In [53]:
metric(Yxm[tr], tr)

(2.869601, 0.748053, 0.374732)


In [54]:
metric(Yxm[cv], cv)

(2.927475, 0.795920, 0.398711)


In [21]:
metric(Y[tr].wmean(W[tr]), cv)

(2.951820, 0.809212, 0.405370)


In [22]:
metric(Y[tr].mean(), cv)

(2.961588, 0.814577, 0.408057)


In [19]:
metric(0, cv)

(3.281396, 1.000000, 0.500944)


In [18]:
metric(Y[cv]*.225, cv)

(2.543082, 0.600625, 0.300879)


### stop

In [ ]:
raise Stop

In [ ]:
k = 2
plt.scatter(Xuse[k], Ya[k]);
plt.plot(Xuse[k], Yx[k]);
plt.xlim(-0.0001, 0.0004);
plt.ylim(-0.0002, 0.0001);

# legit alg

**Big note: passing different length tuples to numba causes multiple compilations for the function!**

In [10]:
from numba import jit
from heapq import *

try:
    del _alg_evaluate_linear
    del _alg_evaluate_affine
except NameError:
    pass

@jit(nopython=True)
def _lr_ready(x):
    return np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)

@jit(nopython=True)
def _lr_fit(x, y, w):
    x = _lr_ready(x)
    wsqrt = w ** 0.5
    thing = np.linalg.lstsq(np.expand_dims(wsqrt, 1) * x, wsqrt * y)
    return thing[0]

@jit(nopython=True)
def _lr_predict(x, beta):
    x = _lr_ready(x)
    return x @ beta.astype(x.dtype)

@jit(nopython=True)
def _lr_rss(y, y_, w):
    return ((y_ - y) ** 2 * w).sum()

@jit(nopython=True)
def _alg_evaluate_linear(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, subk):
    cvs = 0.0
    eps = 0.0
    for g in range(m):
        xtrg, ytrg, wtrg, xcvg, ycvg, wcvg, xepg, yepg, wepg = (
            xtr[:, g::m][:, subk], ytr[:, g], wtr[:, g],
            xcv[:, g::m][:, subk], ycv[:, g], wcv[:, g],
            xep[:, g::m][:, subk], yep[:, g], wep[:, g])
        wsqrt = wtrg ** .5
        beta, _1, _2, _3 = np.linalg.lstsq(np.expand_dims(wsqrt, 1) * xtrg, wsqrt * ytrg)
        ycvg_ = xcvg @ beta.astype(xcvg.dtype)
        cvs += ((ycvg_ - ycvg) ** 2 * wcvg).sum()
        yepg_ = xepg @ beta.astype(xepg.dtype)
        eps += ((yepg_ - yepg) ** 2 * wepg).sum()
    return cvs, eps

@jit(nopython=True)
def _alg_evaluate_affine(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, subk):
    cvs = 0.0
    eps = 0.0
    for g in range(m):
        
        xtrg, ytrg, wtrg, xcvg, ycvg, wcvg, xepg, yepg, wepg = (
            xtr[:, g::m][:, subk], ytr[:, g], wtr[:, g],
            xcv[:, g::m][:, subk], ycv[:, g], wcv[:, g],
            xep[:, g::m][:, subk], yep[:, g], wep[:, g])
        beta = _lr_fit(xtrg, ytrg, wtrg)
        ycvg_ = _lr_predict(xcvg, beta)
        cvs += _lr_rss(ycvg, ycvg_, wcvg)
        yepg_ = _lr_predict(xepg, beta)
        eps += _lr_rss(yepg, yepg_, wepg)
    return cvs, eps

def alg(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, intercept=True):
    '''
    xtr.shape = (ntr, (m, k))
    ytr.shape = wtr.shape = (ntr, m)
    similar for cv, ep
    should not have NaNs (cells corresponding to where w==0 are multiplied by 0 anyway)
    no intercept term is fit on the x side, so make sure for every feature in x, 0 represents no information
    '''
    _alg_evaluate = [_alg_evaluate_linear, _alg_evaluate_affine][intercept]
    k = xtr.columns.get_level_values(0).nunique() # features
    m = xtr.columns.get_level_values(1).nunique() # stocks/groups
    featuresnames = xtr.columns.levels[0]
    features = set(range(k))
    baseline = (ycv ** 2 * wcv).sum().sum(), (yep ** 2 * wep).sum().sum()
    # the mean model uses the ytr and ycv, similar to as if yep was the leaderboard data and ytr and ycv were given
    # some stocks may have total 0 weight, so fillna for those columns
    meandata = (((ytr * wtr).sum() + (ycv * wcv).sum()) / (wtr.sum() + wcv.sum())).fillna(0)
    xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep = (
        xtr.values, (ytr - meandata).values, wtr.values,
        xcv.values, (ycv - meandata).values, wcv.values,
        xep.values, (yep - meandata).values, wep.values)
    meanmodel = (ycv ** 2 * wcv).sum().sum(), (yep ** 2 * wep).sum().sum()
    scores = {(): meanmodel}
    heap = [(meanmodel[0], ())]
    while True:
        minscore0, minfeats = heappop(heap)
        print(minscore0 / baseline[0], scores[minfeats][1] / baseline[1], len(minfeats))
        print('(' + ', '.join(featuresnames[np.array(minfeats)] if minfeats else ()) + ')')
        print()
        print('[{}]'.format(len(features - set(minfeats))), end='')
        for ii, f in enumerate(features - set(minfeats)):
            print('.' if ii % 10 else ii, end='')
            #print('trying', f)
            newfeats = tuple(sorted(minfeats + (f,)))
            if newfeats not in scores:
                newscore = _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep,
                                         m, k, np.array(newfeats, dtype=np.int64))
                scores[newfeats] = newscore
                heappush(heap, (newscore[0], newfeats))
                #print('newguy', (newscore, newfeats))
            #print()
        print()
        if not heap or minscore0 <= heap[0][0]:
            break
    return

### old implementations

#### high level pseudocode

In [ ]:
def alg():
    lastminguy = None
    init(scores)
    init(heap)
    while True:
        minscore, minguy = heap.pop()[1]
        for f in features - minguy:
            newguy = minguy + {f}
            if newguy not in scores:
                score = eval(newguy)
                scores[newguy] = score
                heap += (score, newguy)
        if minscore <= heap[0][0]:
            break

#### naive no numba implementation

In [222]:
def alg(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep):
    groups = xtr.columns.get_level_values(0).unique()
    features = set(xtr.columns.get_level_values(1))
    m, k = len(groups), len(features)
    #
    #
    #@jit(nopython=True)
    def evaluate(feats):
        cvs = []
        eps = []
        for g in groups:
            beta = lr_fit(xtr.loc[:, (g, feats)].values, ytr.loc[:, g].values, wtr.loc[:, g].values)
            y_ = lr_predict(xcv.loc[:, (g, feats)].values, beta)
            cvs += [lr_rss(ycv.loc[:, g].values, y_, wcv.loc[:, g].values)]
            y_ = lr_predict(xep.loc[:, (g, feats)].values, beta)
            eps += [lr_rss(yep.loc[:, g], y_, wep.loc[:, g].values)]
        return sum(cvs), sum(eps)
    #
    baseline, baselinetest = evaluate(())
    scores, scorestest = {(): baseline}, {(): baselinetest}
    heap = [(baseline, ())]
    #
    while True:
        minscore, minfeats = heappop(heap)
        print(minscore / baseline, scorestest[minfeats] / baselinetest, len(minfeats))
        print(minfeats)
        print()
        for f in features - set(minfeats):
            #print('trying', f)
            newfeats = tuple(sorted(minfeats + (f,)))
            if newfeats not in scores:
                newscore, newscoretest = evaluate(newfeats)
                scores[newfeats], scorestest[newfeats] = newscore, newscoretest
                heappush(heap, (newscore, newfeats))
                #print('newguy', (newscore, newfeats))
            #print()
        if not heap or minscore <= heap[0][0]:
            break
    return

#### all numba failed implementation

In [18]:
@jit(nopython=True)
def _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, subk):
    cvs = 0.0
    eps = 0.0
    for g in range(m):
        xtrg, ytrg, wtrg, xcvg, ycvg, wcvg, xepg, yepg, wepg = (
            xtr[:, g*k:(g+1)*k], ytr[:, g*k:(g+1)*k], wtr[:, g*k:(g+1)*k],
            xcv[:, g*k:(g+1)*k], ycv[:, g*k:(g+1)*k], wcv[:, g*k:(g+1)*k],
            xep[:, g*k:(g+1)*k], yep[:, g*k:(g+1)*k], wep[:, g*k:(g+1)*k])
        beta = _lr_fit(xtrg, ytrg, wtrg)
        ycvg_ = _lr_predict(xcvg, ycvg, wcvg)
        cvs += _lr_rss(ycvg, ycvg_, wcvg)
        yepg_ = _lr_predict(xepg, yepg, wepg)
        eps += _lr_rss(yepg, yepg_, wepg)
    return cvs, eps

@jit(nopython=True)
def _alg_help_featsin(scored, feats):
    np.

@jit(nopython=True)
def _alg_help_featsadd(scored, feats):
    pass

@jit(nopython=True)
def _alg_main(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, maxnumfeats, m, k):
    baseline, baselinetest = _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, ())
    scored = np.zeros((k, 1), np.bool)
    #scoreslist = []
    heap = {(baseline, ())}
    minscore = np.inf
    while True:
        newminscore, newminfeats = min(heap)
        heap.remove((newminscore, newminfeats))
        if minscore <= newminscore:
            break
        minscore, minfeats = newminscore, newminfeats
        if len(minfeats) >= maxnumfeats:
            break
        minfeatsset = set(minfeats)
        for f in [f for f in range(k) if f not in minfeatsset]:
            newfeats = tuple(sorted(minfeats + (f,)))
            if newfeats not in scored:
                newscore, newscoretest = _alg_evaluate(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep, m, k, newfeats)
                heap.add((newscore, newfeats))
                scored.add(newfeats)
                #scoreslist.append((newscore, newscoretest, newfeats))
        if not heap:
            break
    return scores[minfeats], scorestest[minfeats], minfeats

def alg_numba(xtr, ytr, wtr, xcv, ycv, wcv, xep, yep, wep):
    groups = xtr.columns.get_level_values(0).unique()
    features = set(xtr.columns.get_level_values(1))
    n, m, k = len(xtr), len(groups), len(features)
    cvs, eps, feats = _alg_main(
        xtr.values, ytr.values, wtr.values, xcv.values, ycv.values, wcv.values, xep.values, yep.values, wep.values,
        40, m, k)
    return cvs, eps, features[feats]

In [19]:
Xtest = Xuse
Ytest = Ya
Wtest = Wa
alg_numba(Xtest[tr], Ytest[tr], Wtest[tr], Xtest[cv], Ytest[cv], Wtest[cv], Xtest[ep], Ytest[ep], Wtest[ep])

TypingError: Failed at nopython (nopython frontend)
Untyped global name 'tuple': cannot determine Numba type of <class 'type'>
File "<ipython-input-18-d2daf27047d8>", line 33